# A basic example - Washington 14-bus grid system

In the following introductory example, the IEEE 14-bus system busses will be used to display the features and capabilities of reXplan. To fully model and calculate the resiliency of specified power grids, three datasets are needed:

- [Power grid data](../modeling/in_grid_modeling.md)
- [Fragility curves](../modeling/in_fragility_curve.md)
- [Hazard definition](../modeling/in_hazard_modeling.md)

To see a more detailed introduction for the input data, see **(I/0) - Input tab** under the 🌐**MODELING** sidebar. For this example grid, the datasets are already provided and for the hazard modeling the methodology of return periods is used.

## Step 1: Library and Data Import

As reXplan utilizes Jupyter Notebook, code sequences can be subdivided into multiple windows.
Common practise is to keep the imports and the initialization seperated, which allows the user to make adjustments to python libraries without re-running the full code.

In [ ]:
# importing reXplan and dependencies
import reXplan as rx
import numpy as np
import warnings
from pandapower.plotting.plotly import pf_res_plotly
warnings.simplefilter("ignore") # warning are ignored for now

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
simulationName = 'ieee_case14';
network = rx.network.Network(simulationName);
simulation = rx.simulation.Sim(simulationName);

In [ ]:
# TODO EXECUTE THIS, HIDDEN IN DOCS
simulationName = 'ieee_case14';
network = rx.network.Network(simulationName);
simulation = rx.simulation.Sim(simulationName);

The classes of network and simulations are shortened to the class [network](../functions/userfunctions.md#network) & [simulation](../functions/userfunctions.md#simulation) for better readability.

### Assess datasets pre-simulation

Since the network is now created, different datasets can be printed. As previously described, the user can now assess the power grid data, fragility curves and hazard and thus verify the input data.

#### [Power Grid (pre run)](../modeling/in_grid_modeling.md)

With the use of the [plotly function of pandapower](https://pandapower.readthedocs.io/en/v2.13.1/plotting/plotly/built-in_plots.html#power-flow-results), the network can be printed.

In [ ]:
pf_res_plotly(network.pp_network);

The spatial information is displayed as provided in the power grid dataset. Power flow is calculated if load profiles are provided and results can be analyzed by hovering over the relating element with the cursor.

#### [Fragility Curve](../modeling/in_fragility_curve.md)

In [ ]:
sns.set_theme(rc={'figure.figsize':(6,5)})
xnew = np.linspace(0, 85, num=100, endpoint=True)
fig, ax = rx.fragilitycurve.plotFragilityCurves(network.fragilityCurves, xnew)

As displayed above, the provided datasets include two different fragility curves for different types of towers. On the horizontal axis, the intensity is displayed as a numerical value in an interval from 0 to 85 with the use of [linspace](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html). The intensity translates to windspeed in m/s. On the vertical axis, the probability of failure is shown as a decimal fraction, which translates to the probability of failure in percentage (0.5 = 50%). 

<div class="alert alert-info">

Note

The fragility curve has to refer to the same intensity type as the hazard model.
</div>

For this example, the intensity type is windspeed measured in m/s.

There is also a possibility to print a fragility curve individually. More information provided under [Fragility Curve](../modeling/in_fragility_curve.md). 

#### [Printing hazards as a return period](../modeling/in_hazard_modeling.md)

The [hazard modeling](../modeling/in_hazard_modeling.md) in this example case uses a [return period](../modeling/in_hazard_modeling.md#method-4-simulate-multiple-events-according-a-given-return-period).

In [ ]:
# printing return periods
sns.set_theme(rc={'figure.figsize':(10,8)})
for rp in network.returnPeriods.keys():
    sns.lineplot(x=network.returnPeriods[rp].x_data, y=network.returnPeriods[rp].y_data, label=rp)
plt.xlabel('Years')
plt.ylabel('Intensity')
plt.show()

As displayed above, the provided datasets include six different return periods (rp1 - rp6). The highest value for given return periods is considered the `reference return period`. On the vertical axis, the intensity is displayed as a numerical value in an interval from 0 to 3.2 in m/s. On the horizontal axis, the return period is shown as a numerical value in years, which can be used to calculate the annual exceedance probability. 


For instance a 200 Years return period has an annual probability of 1/200 = 0.005% [#TODO Terminology](https://hydro-informatics.com/exercises/ex-floods.html?highlight=return#terminology)

#TODO Mathematical Equation here? TEST RETURN PERIODS FOR INTENSITY

<div class="alert alert-info">

Note

The hazard model has to refer to the same intensity type as the fragility curve.

</div>

## Step 2: Running the Simulation

The following command executes the [simulation](../functions/userfunctions.md#simulation) of the return period "rp6" on the previously described fragility curves. This creates an outage schedule for grid elements. Data is stored in `montecarlo_database.csv`.

In [ ]:
simulation.initialize_model_rp(network=network, ref_return_period="rp2", iterationNumber=10, maxStrata=3)

In [ ]:
# EXECUTE THIS, HIDDEN IN DOCS
simulation.initialize_model_rp(network=network, ref_return_period="rp2", iterationNumber=10, maxStrata=3)

In [ ]:
plt.hist(simulation.samples, density=True, bins=20)
for b in np.append(simulation.stratResults["Upper_X1"].values, simulation.stratResults["Lower_X1"].values[0]):
    plt.axvline(x = b, color = 'r')

## Step 3: Running the Optimum Power Flow

Running multiple iterations, to achieve EENS, ELOL
[#TODO Write Paragraph, Calculation of interations (based on init?)]
Creating the engine_database.csv <br>


In [ ]:
simulation.run(network, iterationSet = None, time = None, run_type = 'pm_ac_opf', delta = 1e-16, saveOutput = True)

In [ ]:
# EXECUTE THIS, HIDDEN IN DOCS
simulation.run(network, iterationSet = None, time = None, run_type = 'pm_ac_opf', delta = 1e-16, saveOutput = True)
# How are the Steps calculated? 4 stratas in initialize, but why 40 steps? (10 Steps each?)

# PUT database in debug!

The network gets updated, printing latest iteration:

### [Power Grid (post run)](../modeling/in_grid_modeling.md)

In [ ]:
pf_res_plotly(network.pp_network);
# How can we select a specific iteration?

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})

df = simulation.failureProbs[simulation.failureProbs['element type']=='Line']
sns.lineplot(data=df, x='event intensity', y='failure probability', hue='power element')
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

In [ ]:
import pandas as pd
from utils import * # pplotting functions
df = pd.read_csv(rx.config.path.engineDatabaseFile(simulationName), index_col = [0, 1, 2, 3, 4])
# -> read database with results, BUT WHAT IS INDEX_COL FOR?

df = filter_non_converged_iterations(df) # filterining non-converged iterations

#### Line Data

In [ ]:
from utils import group_by, invert, get_quantiles_on_iterations
df_line = group_by(df.loc[3], 'sum', 'iteration', 'field', 'type').loc[:,:,'line']
df_line_quantiles = invert(get_quantiles_on_iterations(df_line, [0.05,0.5,0.95]))
df_line = invert(df_line)

In [ ]:
df_montecarlo = pd.read_csv(rx.config.path.engineDatabaseFile(simulationName), index_col = [0, 1, 2, 3, 4])

In [ ]:
import plotly.express as px
px.line(df_line, x=df_line.index, y = 'in_service', color = 'iteration')
# Number of lines in Service!

In [ ]:
px.line(df_line_quantiles, x=df_line_quantiles.index, y = 'in_service', color = 'quantile')
# Quantiles of lines in service

In [ ]:
df_load = group_by(filter(df, type = 'load'), 'sum', 'iteration', 'field', 'type')
df_load_quantiles = invert(get_quantiles_on_iterations(df_load, [0.05, 0.25, 0.5, 0.75, 0.95]))
# df_load = invert(df_load) 
# df_load['loss_of_load_p_percentage'] = (df_load['loss_of_load_p_mw'])/df_load['max_p_mw'] *100
df_load_quantiles['loss_of_load_p_percentage'] = (df_load_quantiles['loss_of_load_p_mw'])/df_load_quantiles['max_p_mw'] *100

In [ ]:
px.line(df_load_quantiles, x=df_load_quantiles.index, y = 'loss_of_load_p_percentage', color = 'quantile')

### MC METRICS?

In [ ]:
df_network_condensed = filter(df, type = 'network').sum(axis = 1) # sum over timesteps

In [ ]:
df_network_condensed_ = invert(df_network_condensed)
px.histogram(df_network_condensed_, x='energy_not_served_mwh', histnorm='probability')

In [ ]:
statistics= df_network_condensed.groupby('field').mean() # average over iterations
EENS = statistics['energy_not_served_mwh']
LOLE = statistics['loss_of_load_p_duration_h']
print(f'EENS : {EENS.round(2)} MWh, LOLE : {LOLE.round(2)} h')

In [ ]:
crt_loss_of_load = 30 
df_loss_of_load = df.loc[:,:,"loss_of_load_p_percentage","network"]
Survivability = pd.DataFrame(1 - (df_loss_of_load > crt_loss_of_load).sum() / df_loss_of_load.index.levels[0].size, columns = ['base case'])

In [ ]:
df_aux = pd.read_csv(rx.config.path.engineDatabaseFile('basic_example_v1'), index_col = [0, 1, 2, 3])
df_loss_of_load_aux = df_aux.loc[:,"loss_of_load_p_percentage","network"]
Survivability['line 10 reinforced'] = 1 - (df_loss_of_load_aux > crt_loss_of_load).sum() / df_loss_of_load_aux.index.levels[0].size
df_aux = pd.read_csv(rx.config.path.engineDatabaseFile('basic_example_v2'), index_col = [0, 1, 2, 3])
df_loss_of_load_aux = df_aux.loc[:,"loss_of_load_p_percentage","network"]
Survivability['line 2 reparing time improved'] = 1 - (df_loss_of_load_aux > crt_loss_of_load).sum() / df_loss_of_load_aux.index.levels[0].size

In [ ]:
px.line(Survivability).update_layout(xaxis_title="time", yaxis_title="Survivability")

In [ ]:
px.line(df_load_quantiles, x=df_load_quantiles.index, y = 'loss_of_load_p_percentage', color = 'quantile')

In [ ]:
import plotly.graph_objects as go
df_line = group_by(filter(df, type = 'line'), 'mean','strata', 'iteration', 'field','id') # mean in this case does not have any effect as the groupying levels are the initial ones
df_line = invert(df_line)
# df_line = df_line.loc[df_line.index > '2022-01-01 12:00:00']

fig = go.Figure() # --> put in a function (?)

ids = df_line['id'].drop_duplicates().to_list()

for id in ids:
    fig.add_trace(go.Violin(x=df_line['id'][df_line['id'] == id],
                            y=df_line['loading_percent'][df_line['id'] == id],
                            name=id,
                            box_visible=False,
                            meanline_visible=True,
                            side='positive',
                            orientation = 'v'
                           )
                 )
fig.update_layout(width=1000, height=500)
fig.show()
